# Full Event Interpretation
FEI allows us to reconstruct a tag (the "pair" of a particle of interest) with the data of the reconstructed particle. We can use it in the end to reconstruct decay channels involving invisible particles (i.e. neutrinos)

In this exercise, FEI will be used to reconsruct a $B^{+}_{tag}$

In [1]:
import basf2 as b2
import modularAnalysis as ma
import fei
from variables import variables as vm 
import variables.collections as vc

Welcome to JupyROOT 6.20/04


In [2]:
filenumber = 0

Use an example file as input

In [3]:
main = b2.Path()
# load input data from mdst/udst file
ma.inputMdst(
    environmentType="default",
    filename=b2.find_file(f"starterkit/2021/1111540100_eph3_BGx0_{filenumber}.root", "examples"),
    path=main,
)

Load the recommended weight files according to:

`$ b2conditionsdb-recommend /group/belle2/users/tenchini/prerelease-05-00-00a/charged/charged_eph3_BGx0_{filenumber}.root`

In [4]:
tag = "analysis_tools_release-04-02"
b2.conditions.globaltags = [tag]

In this example, we are only reconstructing charged B mesons, so it is convenient to select only those channels to speed up the FEI

In [5]:
particles = fei.get_default_channels(
    chargedB = True,
    neutralB = False,
    hadronic = True,
    semileptonic = False,
    baryonic = True
)

Set up FEI configuration specifying the FEI prefix and disabling monitoring (as we are not interested in monitoring the performance)

In [6]:
configuration = fei.FeiConfiguration(
    prefix="FEIv4_2020_MC13_release_04_01_01", monitor=False
)

Now, we need to translate that configuration to a path to be appended to the main path

In [7]:
# Get FEI path
feistate = fei.get_path(particles, configuration)

# Add FEI path to the path to be processed
main.add_path(feistate.path)


    ____ _  _ _    _       ____ _  _ ____ _  _ ___    _ _  _ ___ ____ ____ ___  ____ ____ ___ ____ ___ _ ____ _  _
    |___ |  | |    |       |___ |  | |___ |\ |  |     | |\ |  |  |___ |__/ |__] |__/ |___  |  |__|  |  | |  | |\ |
    |    |__| |___ |___    |___  \/  |___ | \|  |     | | \|  |  |___ |  \ |    |  \ |___  |  |  |  |  | |__| | \|

    Author: Thomas Keck 2014 - 2017
    Please cite my PhD thesis
    
Stage 0: Load FSP particles
Stage 0: PreReconstruct particles:  ['pi+', 'K+', 'p+', 'mu+', 'e+', 'gamma']
Stage 1: PreReconstruct particles:  ['pi0', 'Lambda0', 'J/psi']
Stage 2: PreReconstruct particles:  ['K_S0', 'Sigma+']
[INFO] Ignoring vertex fit because multiple pi0 are not supported yet K_S0:generic_1.
Stage 3: PreReconstruct particles:  ['D0', 'D+', 'D_s+', 'Lambda_c+']
[INFO] Ignoring vertex fit because multiple pi0 are not supported yet D0:generic_2.
[INFO] Ignoring vertex fit because multiple pi0 are not supported yet D0:generic_8.
[INFO] Ignoring vertex fit becaus

After adding the FEI to the path, the tags are added to a particle list called `B+:generic`. Now, we want to add Monte Carlo matching and create variable aliases for the `extraInfos` created by the FEI.

In [8]:
# match reconstructed with MC particles
ma.matchMCTruth("B+:generic", path=main)
# add aliases
vm.addAlias("decayModeID", "extraInfo(decayModeID)")
vm.addAlias("SigProb", "extraInfo(SignalProbability)")

True

In this case, it makes sense to rank the reconstructed B mesons (as the process won't be perfect) to pick the bests for our analysis, which can be done using the variable `extraInfo(SignalProbability)`

In [9]:
# Rank B+ candidates by signal classifier output
ma.rankByHighest(
    particleList="B+:generic",
    variable="extraInfo(SignalProbability)",
    outputVariable="FEIProbabilityRank",
    path=main,
)
vm.addAlias("FEIProbRank", "extraInfo(FEIProbabilityRank)")

True

Finally, we will store these variables (along with some others of interest) in a root file

In [10]:
filename = f"FEI_B_charged_hadronic_{filenumber}.root"
# select interesting variables
b_vars = ["Mbc", "deltaE"]
# add created alises
b_vars += ["decayModeID", "SigProb"]
# add MC variables
b_vars += ["mcErrors", "isSignal"]
# add ranking
b_vars += ["FEIProbRank"]
# store results in a root file
ma.variablesToNtuple(
    decayString="B+:generic", variables=b_vars,
    filename=filename,
    treename="tree",
    path=main
)

In [11]:
# Start the event loop (actually start processing things)
print(f"Storing results in {filename}...")
b2.process(main)

Storing results in FEI_B_charged_hadronic_0.root...
